# Analyse pyBadlands stratigraphic output

If the stratigraphic structure is turned on in the input.xml file, **pyBadlands** produces sedimentary layers recorded by hdf5 files. The stratigraphic layers are defined on a regularly spaced grid and a layer is recorded at each layer time interval given by the user.

Here we show how we can visualise quickly the structure of the stratigraphic layer in an IPython notebook.

In [4]:
import os
import numpy as np
import cmocean as cmo
import matplotlib as mpl
from matplotlib import mlab, cm
import matplotlib.mlab as ml
import matplotlib.pyplot as plt
import matplotlib.colors

# Import the python file (.py) which contains all defined functions
from vis_scripts import stratalArchitecture as strata

# Display plots in SVG format
%config InlineBackend.figure_format = 'svg'

# Display plots in cells
%matplotlib inline

ImportError: No module named vis_scripts

# 1-  Visualize stratigraphic layers on a cross-section

## 1.1- Loading the stratigraphic file

First we need to load the stratigraphic files. The files are located in the **h5/** folder in the simulation main output folder and are named using the following convention:
- `sed.time`T`.p`X`.hdf5`

with T the display time index and X the number of the partition (used in the parallel version). In cases where you ran your simulation in parallel you will also need to give the number of CPUs used (_cpus_).

To load a file you will need to give the folder path and the number of processors used in your simulation.

In [ ]:
# For more information regarding the function uncomment the following line.
# help(strata.stratalSection.__init__)

In [ ]:
# folder = 'case3_aus/output/h5/'  # output folder path
folder = 'case3_aus/output_/h5/'  # output folder path
strat = strata.stratalSection(folder, 1)

Then we need to load a particular output time interval (this is the T parameter in the hdf5 file name convention).

**Note**

This number is not always the number of sedimentary layers for this particular time step as you could have chosen in the input file to have more than 1 sedimentary layer recorded by output interval!

In [ ]:
# help(strat.loadStratigraphy)
# help(strat.loadTIN)

In [ ]:
timestep = 140
strat.loadStratigraphy(timestep)  # load strata files
strat.loadTIN(timestep)  # load TIN files

**Important** 

If you want to change the timestep, you need to restart this script (in the top menu, Kernel->Restart) and run from the first cell.

## 1.2- Building a cross-section

To build a cross-section to visualise the stratigraphic layers, you will need to provide:

+ the coordinates of two points deliminating the cross-section **_(x1,y1)_** and **_(x2,y2)_**, 
+ the number of nodes that defines the resolution of this cross-section **_nbpts_** and
+ a gaussian filter value to smooth the the stratigraphic layer (**_gfilt_** a value of 0 can be used for non-smoothing).

Plotting the topography map from the model output can help you to define the location of the cross-section.

In [ ]:
# help(strat.plotSectionMap)

In [ ]:
strat.plotSectionMap(title='Topography map', xlegend='Distance (m)', ylegend='Distance (m)', 
                     color=cmo.cm.delta, crange=[-2000,2000], cs=None, size=(6,6))

In [ ]:
# Coordinates [x,y] of two points on the cross-section
cs=np.zeros((2,2))
cs[0,:] = [2137110.46715,7087591.94151]  # point 1
cs[1,:] = [-112889.532847,7087591.94151]  # point 2

# Interpolation parameters
nbpts = 500  
gfilt = 2  

# Show the location of the cross-section on the topography map
strat.plotSectionMap(title='Topography map', xlegend='Distance (m)', ylegend='Distance (m)',
                     color=cmo.cm.delta, colorcs='magenta', crange=[-2000,2000], cs=cs, size=(6,6))

In [ ]:
# Build cross-section
strat.buildSection(xo = cs[0,0], yo = cs[0,1], xm = cs[1,0], ym = cs[1,1], pts = nbpts, gfilter = gfilt)

## 1.3- Visualize stratal stacking pattern coloured by time

First, we use **plotly** to visualise the vertival cross-section of stratal stacking pattern.

In [ ]:
strata.viewSection(width = 800, height = 500, cs = strat, 
            dnlay = 2, rangeX=[2000, 10000], rangeY=[-400,200],
            linesize = 0.5, title='Stratal stacking pattern coloured by time')

## 1.4- Visualize stratal stacking pattern coloured by facies

First we build paleo-depositional environment (facies) structure based on the paleo-water depth. For example ([reference](https://opentextbc.ca/geology/chapter/6-3-depositional-environments-and-sedimentary-basins/)),

<img src="images/depo-envi.png" alt="depositional environments" width="900" height="500"/>

In [ ]:
# Specify the range of water depth for the depositional environments, see the table above
depthID = [0, -25, -100, -200, -500]

# Define colors for depositional environments, with number of colors equals to len(depthID) + 2
colorDepoenvi = ['white','limegreen','darkkhaki','sandybrown','khaki','c','teal'] 
# 'White' colors where either no deposition or deposited sediemnt thickness < 0.01 m.

# Build an array of depositional environment ID (enviID)
enviID = np.zeros((strat.nz, len(strat.dist)))
enviID = strata.buildEnviID(cs = strat, depthID = depthID)

In [ ]:
# Plot stratal stacking pattern colored by paleo-depositional environments
# It can take up to 5 mins...
# Need to be careful with dnlay in the following line.
strata.viewDepoenvi(width = 8, height = 5, cs = strat, enviID = enviID, dnlay = 2, color = colorDepoenvi, 
                    rangeX=[2000, 12000], rangeY=[-500,100], savefig = 'Yes', figname = 'delta_strata_depoenvi')

# 2-  Build a Wheeler diagram

Wheeler diagram (or chronostratigraphic chart) is a powerful tool to document unconformities between sequences, and to understand the evolution of sedimentary stacking patterns and their relationships to sea level. It displays the horizontal distribution of contemporaneous sedimentary layer sequences, as well as hiatuses in sedimentation.

In [ ]:
# Time structure of the model, corresponding to the Time structure in the input.xml file
start_time = 0.  # the start time of the model run [a]
disptime = 50000.  # the layer interval of the strata module [a]
end_time = start_time + disptime * timestep  # the time of the loaded output [a]
layertime = np.linspace(start_time,end_time,strat.nz)  # time of the layers

# Plot Wheeler diagram
strata.viewWheeler(width = 7, height = 4, cs = strat, enviID = enviID, time = layertime, dnlay = 3, color = colorDepoenvi, 
                   rangeX=[2000, 12000], rangeY = None, savefig = 'Yes', figname = 'delta_Wheeler_diagram')

# 3-  Extract synthetic cores

To plot the synthetic cores (vertical stacking patterns) at any locations on the cross-section, you need to give the location of the core.

In [ ]:
# Location of the core on the cross-section (m)
posit = 7000

# Plot the core
strata.viewCore(width = 2, height = 5, cs = strat, enviID = enviID, posit = posit, time = layertime, 
                color = colorDepoenvi, rangeX = None, rangeY = None, savefig = 'Yes', figname = 'delta_core')